# Part 3 — Meteo Pipeline (REST API Integration)

This notebook demonstrates how to integrate **external REST API data** into an ETL pipeline.

**Pipeline flow:**
1. Read city coordinates (lat/lon) from `gen_zipcode_dim` in MySQL
2. Call the **Open-Meteo API** to fetch hourly temperature forecasts
3. Store raw forecast data in `meteo_eae`
4. Compute average forecast temperature per city as a KPI
5. Write KPI to `gen_kpi_ft`

> **Open-Meteo** (`https://api.open-meteo.com`) is a free, no-auth weather API that returns JSON data.

In [ ]:
import time
import tracemalloc
import yaml
import requests
import pandas as pd
from datetime import date
from pathlib import Path

from utils import get_db_engine, read_from_database, write_to_database

# Start performance tracking
tracemalloc.start()
start_time = time.time()

# Configuration
BASE_DIR = Path.cwd()
CREDS_PATH = Path("C:/Users/roger.lloret/Documents/creds/creds_dmbi.yml")
KPI_TABLE = "gen_kpi_ft"
METEO_TABLE = "meteo_eae"

# Cities to query — must match province names in gen_zipcode_dim
CITIES = ["Madrid", "Barcelona"]

# Open-Meteo API base URL
OPEN_METEO_URL = "https://api.open-meteo.com/v1/forecast"

## Helper Functions

| Function | Purpose |
|----------|---------|
| `get_city_coordinates` | SQL query to get avg lat/lon per province from `gen_zipcode_dim` |
| `fetch_forecast` | Calls Open-Meteo REST API and returns a DataFrame of hourly temperatures |

In [ ]:
def get_city_coordinates(engine, cities: list[str]) -> pd.DataFrame:
    """
    Reads lat/lon from gen_zipcode_dim for the given cities (matched by province).
    Returns one row per city with columns: city, latitude, longitude.
    """
    placeholders = ", ".join(f"'{c}'" for c in cities)
    query = f"""
        SELECT province AS city,
               AVG(zipcode_latitude)  AS latitude,
               AVG(zipcode_longitude) AS longitude
        FROM eae.gen_zipcode_dim
        WHERE province IN ({placeholders})
        GROUP BY province
    """
    return read_from_database(engine, query)


def fetch_forecast(city: str, latitude: float, longitude: float) -> pd.DataFrame:
    """
    Calls the Open-Meteo API for a single location and returns a DataFrame
    with columns: city, datetime, temperature_2m.
    """
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "hourly": "temperature_2m",
    }
    response = requests.get(OPEN_METEO_URL, params=params, timeout=30)
    response.raise_for_status()
    data = response.json()
    hourly = data["hourly"]
    df = pd.DataFrame({
        "city": city,
        "datetime": pd.to_datetime(hourly["time"]),
        "temperature_2m": hourly["temperature_2m"],
    })
    return df

## 1. Load Credentials & Get City Coordinates

In [ ]:
with open(CREDS_PATH, "r") as f:
    creds = yaml.safe_load(f)

db_engine = get_db_engine(creds["data_warehouse"])

# Get coordinates from the database
coords_df = get_city_coordinates(db_engine, CITIES)
print("City coordinates:")
print(coords_df.to_string(index=False))

## 2. Fetch Temperature Forecasts from Open-Meteo API

For each city, we call the REST API with `latitude`, `longitude`, and `hourly=temperature_2m`.

In [ ]:
forecast_frames = []
for _, row in coords_df.iterrows():
    city_forecast = fetch_forecast(row["city"], row["latitude"], row["longitude"])
    forecast_frames.append(city_forecast)
    print(f"  {row['city']}: {len(city_forecast)} hourly records retrieved.")

meteo_df = pd.concat(forecast_frames, ignore_index=True)
print(f"\nTotal forecast rows: {len(meteo_df)}")
meteo_df.head()

## 3. Store Raw Forecast in Database

Write all forecast rows to `meteo_eae` (truncate + reload strategy with `if_exists="replace"`).

In [ ]:
print(f"Writing {len(meteo_df)} rows to {METEO_TABLE}...")
write_to_database(db_engine, meteo_df, METEO_TABLE, if_exists="replace")
print("Done.")

## 4. Build KPI — Average Forecast Temperature per City

In [ ]:
kpi_df = (
    meteo_df
    .groupby("city")["temperature_2m"]
    .mean()
    .reset_index()
)
kpi_df["kpi_name"] = "Average forecast temperature (°C) for " + kpi_df["city"]
kpi_df = kpi_df.rename(columns={"temperature_2m": "kpi_value"})
kpi_df["kpi_date"] = date.today()

output_df = kpi_df[["kpi_date", "kpi_name", "kpi_value"]]
print(output_df.to_string(index=False))

## 5. Write KPI to Database

In [ ]:
print(f"Writing KPIs to {KPI_TABLE}...")
write_to_database(db_engine, output_df, KPI_TABLE)
print("Meteo ETL completed successfully.")

## Performance Metrics

In [ ]:
elapsed = time.time() - start_time
_, peak_memory = tracemalloc.get_traced_memory()
tracemalloc.stop()
print(f"Meteo pipeline executed in {elapsed:.2f} seconds.")
print(f"Peak RAM usage: {peak_memory / (1024 * 1024):.2f} MB")